In [1]:
import argparse
import os
import platform
import shutil
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords, 
    xyxy2xywh, plot_one_box, strip_optimizer, set_logging)
from utils.torch_utils import select_device, load_classifier, time_synchronized



In [2]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--weights', nargs='+', type=str, default='/home/jetson/code/yolov5/yolov5-3.0/1030.pt', help='model.pt path(s)')
# parser.add_argument('--source', type=str, default='0', help='source')  # file/folder, 0 for webcam
# parser.add_argument('--output', type=str, default='inference/output', help='output folder')  # output folder
# parser.add_argument('--img-size', type=int, default=320, help='inference size (pixels)')
# parser.add_argument('--conf-thres', type=float, default=0.4, help='object confidence threshold')
# parser.add_argument('--iou-thres', type=float, default=0.3, help='IOU threshold for NMS')
# parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
# parser.add_argument('--view-img', action='store_true', help='display results')
# parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
# parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --class 0, or --class 0 2 3')
# parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
# parser.add_argument('--augment', action='store_true', help='augmented inference')
# parser.add_argument('--update', action='store_true', help='update all models')
# opt = parser.parse_args()
# print(opt)


In [3]:
def detect(save_img=False):
    out, source, weights, view_img, save_txt, imgsz = 'inference/output', '0', '/home/jetson/code/yolov5/yolov5-3.0/1030.pt', True, True, 320
    webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')

    # Initialize
    set_logging()
    device = select_device('')
    if os.path.exists(out):
        shutil.rmtree(out)  # delete output folder
    os.makedirs(out)  # make new output folder
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages(source, img_size=imgsz)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    
    
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=True)[0]

        # Apply NMS
        pred = non_max_suppression(pred, 0.4, 0.6, classes=None, agnostic=True)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0 = path[i], '%g: ' % i, im0s[i].copy()
            else:
                p, s, im0 = path, '', im0s

            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if int(t1) % 5 ==0:
                print(det)
            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))


    print('Done. (%.3fs)' % (time.time() - t0))

                

In [4]:
detect()

Using CUDA device0 _CudaDeviceProperties(name='NVIDIA Tegra X1', total_memory=3956MB)



Fusing layers... 


Model Summary: 140 layers, 7.24652e+06 parameters, 6.61683e+06 gradients


1/1: 0...  success (640x480 at 30.00 FPS).

0: 256x320 Done. (3.448s)
0: 256x320 Done. (0.292s)
0: 256x320 Done. (0.236s)
0: 256x320 Done. (0.217s)
0: 256x320 Done. (0.241s)
0: 256x320 Done. (0.203s)
0: 256x320 Done. (0.194s)
0: 256x320 Done. (0.194s)
0: 256x320 Done. (0.202s)
0: 256x320 Done. (0.213s)
0: 256x320 Done. (0.200s)
0: 256x320 Done. (0.193s)
0: 256x320 Done. (0.209s)
0: 256x320 Done. (0.209s)
0: 256x320 Done. (0.228s)
0: 256x320 Done. (0.194s)
0: 256x320 Done. (0.214s)
0: 256x320 Done. (0.190s)
0: 256x320 Done. (0.198s)
0: 256x320 Done. (0.194s)
0: 256x320 Done. (0.208s)
0: 256x320 Done. (0.194s)
0: 256x320 Done. (0.193s)
0: 256x320 Done. (0.204s)
0: 256x320 Done. (0.202s)
0: 256x320 Done. (0.208s)
0: 256x320 Done. (0.196s)
0: 256x320 Done. (0.200s)
0: 256x320 Done. (0.193s)
0: 256x320 Done. (0.200s)
0: 256x320 Done. (0.213s)
0: 256x320 Done. (0.198s)
0: 256x320 Done. (0.193s)
0: 256x320 Done. (0.205s)
0: 256x320 Done. (0.205s)
0: 256x320 Done. (0.194s)
0: 256x320 Done. (0.

KeyboardInterrupt: 